In [ ]:
# ============================================================
# EXPERIMENT CELL (HIGH node only) — Day-Ahead (H=100)
# Improvements included:
# - Balanced config (runtime + robustness)
# - Clear HIGH-node selection printout
# - Day-ahead evaluation on HOLDOUT for: Persistence + Recursive (Ridge/RF/HGB) + DirectStacked (best model)
# - Horizon-bucket MAE + Skill score vs Persistence
# - Direct-Stacked CV split by ORIGIN (not by stacked row order)
# - Robust logging for skipped folds / effective folds used
# - Save artifacts (csv/json + MAE(h) curves) into reports/forecast_experiments/<timestamp>/
# ============================================================

import warnings, math, time, json
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", message="X does not have valid feature names")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from joblib import dump 

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

from src.config import CLEAN_TS_DIR


# =========================
# Balanced / thesis-tauglich settings
# =========================
TIMESTAMP_COL = "timestamp"
TARGET = "P_MW"

RAW_FEATURES = ["wind_speed_mps", "solar_radiation_Wm2", "temperature_C"]
TARGET_LAGS = [1, 4, 8, 12, 24, 96]

FOURIER_K = 3
PERIOD_STEPS = 96

MIN_COVERAGE = 0.20

H = 100                          # day-ahead horizon (~25h)

FINAL_HOLDOUT_FRAC = 0.20

OUTER_SPLITS = 3
INNER_SPLITS = 3
N_ITER = 10
RANDOM_STATE = 42
N_JOBS = 1

ORIGIN_STRIDE = 8                # every 2 hours (15min steps)
MAX_TEST_ORIGINS_PER_FOLD = 400  # cap rollouts during model-selection

# Horizon buckets for reporting
H_BUCKETS = [
    (1, 4),
    (5, 12),
    (13, 24),
    (25, 48),
    (49, 100),
]

# Output folder
RUN_TS = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = Path("reports") / "forecast_experiments" / RUN_TS
OUT_DIR.mkdir(parents=True, exist_ok=True)

# save wining models
LATEST_DIR = Path("reports") / "forecast_experiments" / "_current"
LATEST_DIR.mkdir(parents=True, exist_ok=True)


# =========================
# Helpers
# =========================
def evaluate(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mae = float(mean_absolute_error(y_true, y_pred))
    rmse = float(math.sqrt(mean_squared_error(y_true, y_pred)))
    r2 = float(r2_score(y_true, y_pred))
    return {"MAE": mae, "RMSE": rmse, "R2": r2}

def bucket_means(mae_h, buckets):
    out = {}
    for a, b in buckets:
        # horizons are 1-indexed in naming, but array is 0-indexed
        seg = mae_h[(a-1):b]
        out[f"MAE_h{a:03d}-{b:03d}"] = float(np.nanmean(seg))
    return out

def skill_score(mae_model, mae_baseline):
    return float(1.0 - (mae_model / mae_baseline))

def add_time_features(df):
    ts = pd.to_datetime(df[TIMESTAMP_COL], utc=False)
    df["hour"] = ts.dt.hour
    df["dayofweek"] = ts.dt.dayofweek
    df["month"] = ts.dt.month

    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    df["dow_sin"]  = np.sin(2 * np.pi * df["dayofweek"] / 7)
    df["dow_cos"]  = np.cos(2 * np.pi * df["dayofweek"] / 7)
    df["month_sin"]= np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"]= np.cos(2 * np.pi * df["month"] / 12)
    return df

def add_fourier_features(df, period_steps=PERIOD_STEPS, K=FOURIER_K, prefix="day"):
    step_idx = (np.arange(len(df), dtype=float) % period_steps)
    for k in range(1, K + 1):
        df[f"{prefix}_sin{k}"] = np.sin(2 * np.pi * k * step_idx / period_steps)
        df[f"{prefix}_cos{k}"] = np.cos(2 * np.pi * k * step_idx / period_steps)
    return df

def add_target_lags(df, target_col, lags):
    for L in lags:
        df[f"{target_col}_lag{L}"] = df[target_col].shift(L)
    return df

def prepare_df(df_raw):
    df = df_raw.copy()
    df[TIMESTAMP_COL] = pd.to_datetime(df[TIMESTAMP_COL], utc=False)
    df = df.sort_values(TIMESTAMP_COL).reset_index(drop=True)

    df = add_time_features(df)
    df = add_fourier_features(df)
    df = add_target_lags(df, TARGET, TARGET_LAGS)

    df["target_1"] = df[TARGET].shift(-1)

    present_feats = [c for c in RAW_FEATURES if c in df.columns]
    time_feats = ["hour_sin","hour_cos","dow_sin","dow_cos","month_sin","month_cos"]
    fourier_feats = [f"day_sin{k}" for k in range(1, FOURIER_K+1)] + [f"day_cos{k}" for k in range(1, FOURIER_K+1)]
    lag_feats = [f"{TARGET}_lag{L}" for L in TARGET_LAGS]

    exog_cols = present_feats + time_feats + fourier_feats
    lag_cols = lag_feats
    feature_cols = exog_cols + lag_cols

    df_model = df.dropna(subset=feature_cols + ["target_1", TARGET]).copy().reset_index(drop=True)
    return df_model, feature_cols, exog_cols, lag_cols, present_feats

def make_models_and_spaces():
    models = {}

    models["Ridge"] = (
        Pipeline([("scaler", StandardScaler()), ("model", Ridge(random_state=RANDOM_STATE))]),
        {"model__alpha": np.logspace(-3, 3, 20).tolist()},
    )

    models["RF"] = (
        RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS),
        {
            "n_estimators": [200, 400, 800],
            "max_depth": [None, 8, 12, 16, 24],
            "min_samples_leaf": [1, 2, 5, 10],
            "max_features": ["sqrt", 0.5, 0.8, None],
        },
    )

    models["HGB"] = (
        HistGradientBoostingRegressor(random_state=RANDOM_STATE),
        {
            "learning_rate": [0.03, 0.05, 0.08, 0.1],
            "max_depth": [None, 6, 10, 14],
            "max_leaf_nodes": [31, 63, 127],
            "min_samples_leaf": [10, 20, 50],
            "l2_regularization": [0.0, 0.1, 1.0],
        },
    )

    return models

def _update_lags_from_history(lag_cols, history_values):
    # lag col format: "P_MW_lag96" etc.
    return np.array([history_values[int(col.split("lag")[1])] for col in lag_cols], dtype=float)

def evaluate_day_ahead_recursive(model, df_model, exog_cols, lag_cols, H=100, origin_indices=None, origin_stride=1):
    n = len(df_model)
    max_origin = n - H - 1
    if max_origin <= 0:
        raise ValueError(f"Not enough data for H={H}. n={n}")

    if origin_indices is None:
        origin_indices = np.arange(0, max_origin + 1, origin_stride, dtype=int)
    else:
        origin_indices = np.asarray(origin_indices, dtype=int)
        origin_indices = origin_indices[origin_indices <= max_origin]

    abs_errors_h = [[] for _ in range(H)]
    sq_errors_h = [[] for _ in range(H)]
    y_true_all, y_pred_all = [], []

    lag_steps = [int(c.split("lag")[1]) for c in lag_cols]
    max_lag = max(lag_steps)

    for j in origin_indices:
        # history[L] stores y(t-L+1) with L=1 being current y(t)
        history = {}
        for L in range(1, max_lag + 1):
            idx = j - (L - 1)
            history[L] = float(df_model[TARGET].iloc[idx]) if idx >= 0 else np.nan
        if any(np.isnan(history[L]) for L in lag_steps):
            continue

        for k in range(1, H + 1):
            step_idx = j + (k - 1)  # feature time
            exog = df_model.loc[step_idx, exog_cols].astype(float).values if exog_cols else np.array([], dtype=float)
            lag_vec = _update_lags_from_history(lag_cols, history)
            x = np.concatenate([exog, lag_vec]).reshape(1, -1)

            y_hat = float(model.predict(x)[0])
            y_true = float(df_model[TARGET].iloc[j + k])

            err = y_true - y_hat
            abs_errors_h[k-1].append(abs(err))
            sq_errors_h[k-1].append(err * err)

            y_true_all.append(y_true)
            y_pred_all.append(y_hat)

            # shift lags and add prediction
            for L in range(max_lag, 1, -1):
                history[L] = history[L-1]
            history[1] = y_hat

    mae_h = np.array([np.mean(v) if len(v) else np.nan for v in abs_errors_h], dtype=float)
    rmse_h = np.array([math.sqrt(np.mean(v)) if len(v) else np.nan for v in sq_errors_h], dtype=float)
    overall = evaluate(np.array(y_true_all), np.array(y_pred_all))
    return overall, mae_h, rmse_h

def evaluate_day_ahead_persistence(df_model, H=100, origin_stride=1):
    n = len(df_model)
    max_origin = n - H - 1
    origin_indices = np.arange(0, max_origin + 1, origin_stride, dtype=int)

    abs_errors_h = [[] for _ in range(H)]
    sq_errors_h = [[] for _ in range(H)]
    y_true_all, y_pred_all = [], []

    for j in origin_indices:
        y0 = float(df_model[TARGET].iloc[j])
        for k in range(1, H + 1):
            y_true = float(df_model[TARGET].iloc[j + k])
            y_hat = y0
            err = y_true - y_hat
            abs_errors_h[k-1].append(abs(err))
            sq_errors_h[k-1].append(err * err)
            y_true_all.append(y_true)
            y_pred_all.append(y_hat)

    mae_h = np.array([np.mean(v) if len(v) else np.nan for v in abs_errors_h], dtype=float)
    rmse_h = np.array([math.sqrt(np.mean(v)) if len(v) else np.nan for v in sq_errors_h], dtype=float)
    overall = evaluate(np.array(y_true_all), np.array(y_pred_all))
    return overall, mae_h, rmse_h

# ---------- Direct (stacked) dataset + CV by ORIGIN ----------
def build_stacked_direct_dataset(df_model, exog_cols, lag_cols, H=100, origin_stride=8):
    n = len(df_model)
    max_origin = n - H - 1
    origins = np.arange(0, max_origin + 1, origin_stride, dtype=int)

    X_rows, y_rows, origin_ids = [], [], []

    for j in origins:
        lag_vec = df_model.loc[j, lag_cols].astype(float).values
        if np.any(np.isnan(lag_vec)):
            continue

        for h in range(1, H + 1):
            tgt_idx = j + h
            exog_vec = df_model.loc[tgt_idx, exog_cols].astype(float).values if exog_cols else np.array([], dtype=float)
            h_feat = np.array([h, h / H], dtype=float)
            X_rows.append(np.concatenate([exog_vec, lag_vec, h_feat]))
            y_rows.append(float(df_model[TARGET].iloc[tgt_idx]))
            origin_ids.append(j)

    Xs = np.asarray(X_rows, dtype=float)
    ys = np.asarray(y_rows, dtype=float)
    origin_ids = np.asarray(origin_ids, dtype=int)
    return Xs, ys, origin_ids

def make_origin_time_cv_splits(origin_ids, n_splits=3):
    """
    Build sklearn-compatible CV splits (train_idx, test_idx) where splits happen on ORIGIN level.
    This avoids leakage in stacked (t,h) rows by ensuring validation origins are strictly later.
    """
    unique_origins = np.unique(origin_ids)
    unique_origins.sort()

    tss = TimeSeriesSplit(n_splits=n_splits)
    splits = []
    for tr_o_idx, te_o_idx in tss.split(unique_origins):
        tr_origins = set(unique_origins[tr_o_idx])
        te_origins = set(unique_origins[te_o_idx])

        tr_idx = np.where(np.isin(origin_ids, list(tr_origins)))[0]
        te_idx = np.where(np.isin(origin_ids, list(te_origins)))[0]
        splits.append((tr_idx, te_idx))
    return splits

def evaluate_day_ahead_direct(model, df_model, exog_cols, lag_cols, H=100, origin_stride=8):
    n = len(df_model)
    max_origin = n - H - 1
    origin_indices = np.arange(0, max_origin + 1, origin_stride, dtype=int)

    abs_errors_h = [[] for _ in range(H)]
    sq_errors_h = [[] for _ in range(H)]
    y_true_all, y_pred_all = [], []

    for j in origin_indices:
        lag_vec = df_model.loc[j, lag_cols].astype(float).values
        if np.any(np.isnan(lag_vec)):
            continue

        X_batch, y_true_batch = [], []
        for h in range(1, H + 1):
            tgt_idx = j + h
            exog_vec = df_model.loc[tgt_idx, exog_cols].astype(float).values if exog_cols else np.array([], dtype=float)
            h_feat = np.array([h, h / H], dtype=float)
            X_batch.append(np.concatenate([exog_vec, lag_vec, h_feat]))
            y_true_batch.append(float(df_model[TARGET].iloc[tgt_idx]))

        X_batch = np.asarray(X_batch, dtype=float)
        y_true_batch = np.asarray(y_true_batch, dtype=float)

        y_hat_batch = model.predict(X_batch).astype(float)

        for h in range(1, H + 1):
            err = y_true_batch[h-1] - y_hat_batch[h-1]
            abs_errors_h[h-1].append(abs(err))
            sq_errors_h[h-1].append(err * err)
            y_true_all.append(y_true_batch[h-1])
            y_pred_all.append(y_hat_batch[h-1])

    mae_h = np.array([np.mean(v) if len(v) else np.nan for v in abs_errors_h], dtype=float)
    rmse_h = np.array([math.sqrt(np.mean(v)) if len(v) else np.nan for v in sq_errors_h], dtype=float)
    overall = evaluate(np.array(y_true_all), np.array(y_pred_all))
    return overall, mae_h, rmse_h


# =========================
# 0) Select ONLY the single HIGH node (max abs(median(P_MW)))
# =========================
meas_dir = Path(CLEAN_TS_DIR)
files = sorted(meas_dir.glob("*_hist_clean.csv"))
assert files, f"Keine *_hist_clean.csv in {meas_dir}"

rows = []
for p in files:
    node_id = p.stem.replace("_hist_clean", "")
    df = pd.read_csv(p, parse_dates=[TIMESTAMP_COL]).sort_values(TIMESTAMP_COL)
    if TARGET not in df.columns:
        continue
    s = df[TARGET].astype(float)

    coverage = 1.0 - s.isna().mean()
    if coverage < MIN_COVERAGE:
        continue

    med = float(s.median(skipna=True))
    rows.append({
        "node_id": node_id,
        "coverage": float(coverage),
        "mean": float(s.mean(skipna=True)),
        "median": med,
        "abs_median": abs(med),
        "p95": float(s.quantile(0.95)),
        "std": float(s.std(skipna=True)),
        "n_raw": int(len(s)),
    })

stats = pd.DataFrame(rows)
assert not stats.empty, "Nach Coverage-Filter keine Nodes übrig. MIN_COVERAGE senken."

stats = stats.sort_values("abs_median").reset_index(drop=True)
high_node = stats.iloc[-1]["node_id"]

print("Auswahlkriterium: MAX abs(median(P_MW)) mit MIN_COVERAGE", MIN_COVERAGE)
print("HIGH node:", high_node)
print("HIGH node stats:")
display(stats[stats["node_id"] == high_node])

stats.to_csv(OUT_DIR / "node_stats.csv", index=False)


# =========================
# 1) Load and prepare data for HIGH node
# =========================
path = meas_dir / f"{high_node}_hist_clean.csv"
df_raw = pd.read_csv(path)

df_model, feature_cols, exog_cols, lag_cols, present_feats = prepare_df(df_raw)

print(f"\nPrepared supervised base for {high_node}: n={len(df_model)}")
print("Exog present:", present_feats)
print("Feature cols:", len(feature_cols), "(exog/time/fourier:", len(exog_cols), ", lags:", len(lag_cols), ")")
assert len(df_model) > (H + 500), f"Time series too short for H={H}. n={len(df_model)}"


# =========================
# 2) Define train/holdout split
# =========================
n = len(df_model)
split_idx = int((1.0 - FINAL_HOLDOUT_FRAC) * n)

holdout_start = split_idx
holdout_max_origin_global = n - H - 1
if holdout_start > holdout_max_origin_global:
    raise ValueError("Holdout window too small for day-ahead evaluation. Reduce FINAL_HOLDOUT_FRAC or H.")

df_train_full = df_model.iloc[:split_idx].reset_index(drop=True)
df_hold = df_model.iloc[holdout_start:].reset_index(drop=True)

print(f"\nFinal split: train [0:{split_idx}) | holdout [{split_idx}:{n})")
print(f"Holdout length: {len(df_hold)} | allowed max origin in holdout-only frame: {len(df_hold) - H - 1}")


# =========================
# 3) Baseline: Persistence on HOLDOUT (same slicing/stride as models)
# =========================
baseline_overall, baseline_mae_h, baseline_rmse_h = evaluate_day_ahead_persistence(
    df_model=df_hold,
    H=H,
    origin_stride=ORIGIN_STRIDE
)


# =========================
# 4) Model selection on TRAIN (tune one-step; select by day-ahead recursive on outer folds)
# =========================
models = make_models_and_spaces()

X_train_full = df_train_full[feature_cols].astype(float).reset_index(drop=True)
y_train_full = df_train_full["target_1"].astype(float).reset_index(drop=True)

outer = TimeSeriesSplit(n_splits=OUTER_SPLITS)
cv_rows = []
fold_skips = []

print("\n" + "="*90)
print("MODEL SELECTION (TRAIN): tune one-step, evaluate day-ahead recursive on outer folds")
print("="*90)

for model_name, (estimator, param_dist) in models.items():
    fold_scores = []
    fold_fit_secs = []
    used_folds = 0

    for fold_i, (tr_idx, te_idx) in enumerate(outer.split(X_train_full), start=1):
        X_tr, y_tr = X_train_full.iloc[tr_idx], y_train_full.iloc[tr_idx]

        inner = TimeSeriesSplit(n_splits=INNER_SPLITS)
        search = RandomizedSearchCV(
            estimator=estimator,
            param_distributions=param_dist,
            n_iter=N_ITER,
            scoring="neg_mean_absolute_error",
            cv=inner,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS,
            verbose=0,
        )

        t0 = time.time()
        search.fit(X_tr, y_tr)
        fit_s = time.time() - t0

        best = search.best_estimator_

        # Fold dataframe: concatenate train+test portion (for recursion start), then restrict origins to test-only
        df_fold = df_train_full.iloc[np.r_[tr_idx, te_idx]].reset_index(drop=True)
        test_start = len(tr_idx)
        test_end = len(tr_idx) + len(te_idx) - 1

        max_origin = len(df_fold) - H - 1
        origin_start = test_start
        origin_end = min(max_origin, test_end - H)

        if origin_end < origin_start:
            fold_skips.append({"model": model_name, "outer_fold": fold_i, "reason": "test_window_too_short_for_H"})
            continue

        origins = np.arange(origin_start, origin_end + 1, ORIGIN_STRIDE, dtype=int)
        if MAX_TEST_ORIGINS_PER_FOLD is not None and len(origins) > MAX_TEST_ORIGINS_PER_FOLD:
            origins = origins[:MAX_TEST_ORIGINS_PER_FOLD]

        overall, mae_h, rmse_h = evaluate_day_ahead_recursive(
            model=best,
            df_model=df_fold,
            exog_cols=exog_cols,
            lag_cols=lag_cols,
            H=H,
            origin_indices=origins,
            origin_stride=ORIGIN_STRIDE
        )

        used_folds += 1
        fold_scores.append(overall)
        fold_fit_secs.append(fit_s)

        cv_rows.append({
            "model": model_name,
            "outer_fold": fold_i,
            "dayahead_MAE": overall["MAE"],
            "dayahead_RMSE": overall["RMSE"],
            "dayahead_R2": overall["R2"],
            "fit_seconds": float(fit_s),
            "best_params": str(search.best_params_),
            "n_test_origins": int(len(origins)),
        })

    # Mean summary row (only over used folds)
    if used_folds > 0:
        cv_rows.append({
            "model": model_name,
            "outer_fold": "MEAN",
            "dayahead_MAE": float(np.mean([s["MAE"] for s in fold_scores])),
            "dayahead_RMSE": float(np.mean([s["RMSE"] for s in fold_scores])),
            "dayahead_R2": float(np.mean([s["R2"] for s in fold_scores])),
            "fit_seconds": float(np.mean(fold_fit_secs)) if fold_fit_secs else np.nan,
            "best_params": "(varies per fold)",
            "n_test_origins": int(np.mean([r["n_test_origins"] for r in cv_rows if r["model"] == model_name and r["outer_fold"] != "MEAN"])) if any((r["model"] == model_name and r["outer_fold"] != "MEAN") for r in cv_rows) else 0,
            "used_folds": int(used_folds),
        })

cv_df = pd.DataFrame(cv_rows)
print("\nOuter-fold day-ahead results (TRAIN backtest):")
display(cv_df)

if fold_skips:
    skip_df = pd.DataFrame(fold_skips)
    print("\nSkipped folds (info):")
    display(skip_df)
    skip_df.to_csv(OUT_DIR / "model_selection_skipped_folds.csv", index=False)

mean_df = cv_df[cv_df["outer_fold"].astype(str) == "MEAN"].sort_values("dayahead_MAE")
print("\nModel ranking by day-ahead MAE (lower is better):")
display(mean_df[["model","dayahead_MAE","dayahead_RMSE","dayahead_R2","fit_seconds","used_folds"]])

best_model_name = mean_df.iloc[0]["model"]
print("\nSelected BEST model by day-ahead MAE:", best_model_name)

cv_df.to_csv(OUT_DIR / "model_selection_outer_results.csv", index=False)


# =========================
# 5) Final tuning on TRAIN (one-step) per model + HOLDOUT day-ahead recursive evaluation
# =========================
final_rows = []
final_curves = {}  # model -> mae_h
final_params = {}

print("\n" + "="*90)
print("FINAL HOLDOUT EVALUATION (DAY-AHEAD, RECURSIVE)")
print("="*90)

for model_name, (estimator, param_dist) in models.items():
    inner = TimeSeriesSplit(n_splits=INNER_SPLITS)
    search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_dist,
        n_iter=max(25, N_ITER),
        scoring="neg_mean_absolute_error",
        cv=inner,
        random_state=RANDOM_STATE,
        n_jobs=N_JOBS,
        verbose=0,
    )

    t0 = time.time()
    search.fit(X_train_full, y_train_full)
    fit_s = time.time() - t0

    best = search.best_estimator_
    final_params[f"recursive_{model_name}"] = search.best_params_

    overall, mae_h, rmse_h = evaluate_day_ahead_recursive(
        model=best,
        df_model=df_hold,
        exog_cols=exog_cols,
        lag_cols=lag_cols,
        H=H,
        origin_indices=None,
        origin_stride=ORIGIN_STRIDE
    )

    final_curves[f"Recursive_{model_name}"] = mae_h

    row = {
        "node": high_node,
        "approach": "Recursive",
        "model": model_name,
        "MAE": overall["MAE"],
        "RMSE": overall["RMSE"],
        "R2": overall["R2"],
        "Skill_vs_Persistence": skill_score(overall["MAE"], baseline_overall["MAE"]),
        "fit_seconds": float(fit_s),
        "best_params": str(search.best_params_),
        **bucket_means(mae_h, H_BUCKETS),
    }
    final_rows.append(row)

final_df = pd.DataFrame(final_rows).sort_values("MAE")
print("\nFinal holdout (recursive) — sorted by MAE:")
display(final_df)

best_recursive_row = final_df.iloc[0].to_dict()  # best by MAE on holdout recursive
final_df.to_csv(OUT_DIR / "final_holdout_recursive_results.csv", index=False)


# =========================
# 6) Direct Multi-Horizon (Stacked) only for BEST model (as selected on TRAIN ranking)
#     - CV split by ORIGIN
# =========================
print("\n" + "="*90)
print(f"DIRECT MULTI-HORIZON (STACKED) for BEST model: {best_model_name}")
print("="*90)

if best_model_name == "Ridge":
    direct_estimator = Pipeline([("scaler", StandardScaler()), ("model", Ridge(random_state=RANDOM_STATE))])
    direct_space = {"model__alpha": np.logspace(-3, 3, 20).tolist()}
elif best_model_name == "RF":
    direct_estimator = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=N_JOBS)
    # already tightened (keep it tighter to avoid explosion)
    direct_space = {
        "n_estimators": [200, 400],
        "max_depth": [None, 12, 24],
        "min_samples_leaf": [1, 5],
        "max_features": ["sqrt", 0.8],
    }
else:
    direct_estimator = HistGradientBoostingRegressor(random_state=RANDOM_STATE)
    direct_space = {
        "learning_rate": [0.03, 0.05, 0.08, 0.1],
        "max_depth": [None, 6, 10, 14],
        "max_leaf_nodes": [31, 63, 127],
        "min_samples_leaf": [10, 20, 50],
        "l2_regularization": [0.0, 0.1, 1.0],
    }

# build stacked dataset from TRAIN
X_direct, y_direct, origin_ids = build_stacked_direct_dataset(
    df_model=df_train_full,
    exog_cols=exog_cols,
    lag_cols=lag_cols,
    H=H,
    origin_stride=ORIGIN_STRIDE
)

print(f"Direct stacked training set: X={X_direct.shape}, y={y_direct.shape}, unique_origins={len(np.unique(origin_ids))}")

direct_cv_splits = make_origin_time_cv_splits(origin_ids, n_splits=INNER_SPLITS)

direct_search = RandomizedSearchCV(
    estimator=direct_estimator,
    param_distributions=direct_space,
    n_iter=max(25, N_ITER),
    scoring="neg_mean_absolute_error",
    cv=direct_cv_splits,           # <-- origin-based CV
    random_state=RANDOM_STATE,
    n_jobs=N_JOBS,
    verbose=0,
)

t0 = time.time()
direct_search.fit(X_direct, y_direct)
direct_fit_s = time.time() - t0

direct_best = direct_search.best_estimator_
final_params["direct_best_model"] = best_model_name
final_params[f"direct_{best_model_name}"] = direct_search.best_params_

direct_overall, direct_mae_h, direct_rmse_h = evaluate_day_ahead_direct(
    model=direct_best,
    df_model=df_hold,
    exog_cols=exog_cols,
    lag_cols=lag_cols,
    H=H,
    origin_stride=ORIGIN_STRIDE
)

dump(
    direct_best, 
    OUT_DIR / f"final_model_direct_{best_model_name.lower()}.joblib"
)


final_curves[f"DirectStacked_{best_model_name}"] = direct_mae_h

direct_summary = {
    "node": high_node,
    "approach": "DirectStacked",
    "model": best_model_name,
    "MAE": direct_overall["MAE"],
    "RMSE": direct_overall["RMSE"],
    "R2": direct_overall["R2"],
    "Skill_vs_Persistence": skill_score(direct_overall["MAE"], baseline_overall["MAE"]),
    "fit_seconds": float(direct_fit_s),
    "best_params": str(direct_search.best_params_),
    **bucket_means(direct_mae_h, H_BUCKETS),
}

# =========================
# 6b) Persist: WINNER model (overwrite "latest")
# =========================

# Save winner model + meta
winner_info = {
    "winner_approach": "DirectStacked",
    "winner_model_name": str(best_model_name),
    "H": int(H),
    "timestamp_col": TIMESTAMP_COL,
    "target_col": TARGET,
    "raw_features": RAW_FEATURES,
    "target_lags": TARGET_LAGS,
    "fourier_k": FOURIER_K,
    "period_steps": PERIOD_STEPS,
    "exog_cols": exog_cols,
    "lag_cols": lag_cols,
    "feature_cols_one_step": feature_cols,
    "direct_h_features": ["h", "h_over_H"],
    "baseline_holdout": baseline_overall,
    "winner_holdout": {
        "MAE": float(direct_summary["MAE"]),
        "RMSE": float(direct_summary["RMSE"]),
        "R2": float(direct_summary["R2"]),
        "Skill_vs_Persistence": float(direct_summary["Skill_vs_Persistence"]),
    },
    "node_used_for_selection": str(high_node),
    "note": "Auto-saved by notebook. Overwritten each run.",
}

dump(direct_best, LATEST_DIR / "best_model.joblib")
(LATEST_DIR / "best_model_meta.json").write_text(
    json.dumps(winner_info, indent=2),
    encoding="utf-8",
)

print("Saved winner model to:", (LATEST_DIR / "best_model.joblib").resolve())
print("Saved winner meta  to:", (LATEST_DIR / "best_model_meta.json").resolve())



# =========================
# 7) Final comparison output (Baseline + Recursive models + Direct best)
# =========================
baseline_summary = {
    "node": high_node,
    "approach": "Baseline",
    "model": "Persistence",
    "MAE": baseline_overall["MAE"],
    "RMSE": baseline_overall["RMSE"],
    "R2": baseline_overall["R2"],
    "Skill_vs_Persistence": 0.0,
    "fit_seconds": 0.0,
    "best_params": "-",
    **bucket_means(baseline_mae_h, H_BUCKETS),
}

final_compare = pd.DataFrame(
    [baseline_summary]
    + final_rows
    + [direct_summary]
).sort_values(["MAE","RMSE"])

print("\n" + "="*90)
print("FINAL COMPARISON (HOLDOUT, DAY-AHEAD H=100)")
print("="*90)
display(final_compare)

final_compare.to_csv(OUT_DIR / "final_comparison.csv", index=False)

# Save curves to CSV (h, each curve)
curves_df = pd.DataFrame({"h": np.arange(1, H+1)})
curves_df["Baseline_Persistence"] = baseline_mae_h
for name, curve in final_curves.items():
    curves_df[name] = curve
curves_df.to_csv(OUT_DIR / "mae_by_horizon_curves.csv", index=False)

# Save run config & params
run_config = {
    "high_node": high_node,
    "n_df_model": int(len(df_model)),
    "n_train": int(len(df_train_full)),
    "n_holdout": int(len(df_hold)),
    "H": H,
    "FINAL_HOLDOUT_FRAC": FINAL_HOLDOUT_FRAC,
    "OUTER_SPLITS": OUTER_SPLITS,
    "INNER_SPLITS": INNER_SPLITS,
    "N_ITER": N_ITER,
    "ORIGIN_STRIDE": ORIGIN_STRIDE,
    "MAX_TEST_ORIGINS_PER_FOLD": MAX_TEST_ORIGINS_PER_FOLD,
    "feature_cols": feature_cols,
    "exog_cols": exog_cols,
    "lag_cols": lag_cols,
    "present_exog": present_feats,
    "h_buckets": H_BUCKETS,
}
with open(OUT_DIR / "run_config.json", "w") as f:
    json.dump(run_config, f, indent=2)

with open(OUT_DIR / "best_params.json", "w") as f:
    json.dump(final_params, f, indent=2)

print(f"\nSaved artifacts to: {OUT_DIR.resolve()}")


# -------------------------
# Plot: MAE(h) curves
# -------------------------
plt.figure(figsize=(12, 5))
plt.plot(np.arange(1, H+1), baseline_mae_h, label="Baseline: Persistence")

for r in final_rows:
    key = f"Recursive_{r['model']}"
    plt.plot(np.arange(1, H+1), final_curves[key], label=f"Recursive: {r['model']}")

plt.plot(np.arange(1, H+1), direct_mae_h, linewidth=2.5, label=f"DirectStacked: {best_model_name}")

plt.title(f"Day-Ahead Error vs Horizon (MAE(h)) — Node: {high_node} — H={H}")
plt.xlabel("Horizon h (steps ahead)")
plt.ylabel("MAE")
plt.grid(True, alpha=0.25)
plt.legend()
plt.tight_layout()
plt.show()

print("\nBest model (TRAIN ranking):", best_model_name)
print("Baseline (holdout):", baseline_overall)
print("DirectStacked (holdout):", {k: direct_summary[k] for k in ["MAE","RMSE","R2","Skill_vs_Persistence"]})
